# Experiment: _Combined text-/graph features vs. text-only features_

In [ ]:
from notebook_prelude import *

In [ ]:
EXPERIMENT_NAME = 'graph_combined'
experiment_data = experiment_helper.get_experiment_config_for(EXPERIMENT_NAME)
param_grid = experiment_data['params_per_type']
df = results_helper.get_results(filter_out_experiment=EXPERIMENT_NAME, filter_out_non_complete_datasets=False).fillna('(MISSING)')
pipeline_helper.remove_complex_types(pipeline_helper.flatten_nested_params(param_grid))

In [ ]:
VECTORIZER_TFIDF = 'TfidfVectorizer'
VECTORIZER_COUNT = 'CountVectorizer'

In [ ]:
def only_types(df, types):
    mask = np.zeros(len(df), dtype=bool)
    for t in types: mask |= (df['type'] == t)
    return mask

df[only_types(df, [TYPE_CONCEPT_MAP, TYPE_COOCCURRENCE, 'text']) & (df.text__vectorizer != VECTORIZER_TFIDF)].groupby(['dataset', 'combined', 'type']).mean_test_f1_macro.max().to_frame().unstack().unstack()

In [ ]:
for index, df_ in df.groupby(['dataset', 'type', 'text__vectorizer', 'combined', 'classifier__C', 'text__vectorizer__binary']):
    if len(df_) != 1:
        display(df_)
        break